In [ ]:
# Data Extraction & Processing

import pandas as pd
import numpy as np
import calendar
from math import ceil
import datetime as dt
from datetime import timedelta
import time
import os
import shutil
import zipfile
import sys
import logging
from dateutil.relativedelta import relativedelta
from subprocess import call
from functools import reduce

# define pathsM

path_new='D:/Roster/'
# path to actual rosters
Roster2016=path_new+'Roster 2016/'
Roster2017=path_new+'Roster 2017/'
Roster2018=path_new+'Roster 2018/'
# path to historical roster assignments
RosterH2016=path_new+'RosterH 2016/'
RosterH2017=path_new+'RosterH 2017/'
RosterH2018=path_new+'RosterH 2018/'
#path to publish date file
SBYRequired=path_new+'SBY Required/'
# path to pairing files
path='D:/'
os.chdir(path)
# path to log file
DataFolder = 'D:/Logs/'
ProcessLogFileName = DataFolder + 'Eventlog.log'


def weekOfMonth(date1):
    first_day = date1.replace(day=1)
    dom = date1.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))
    

""" Returns if date is 'SAT' or 'SUN'"""  

def isWeekend(date1):
    if(date1.isoweekday() == 6):
        return 1
    elif(date1.isoweekday() == 7):
        return 1
    else:
        return 0

""" Get SS33,SS55,SS77,SS88 pairing data """
def PlannedRoster(strfilepath,start_date,end_date):

    roster= np.genfromtxt(strfilepath, delimiter=[1,8,3,3,8,4,8,8,4,5,1,2,8,6,9,3,20],usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,15,16),skip_header= True,skip_footer=True,dtype=str)

    roster= pd.DataFrame(roster, columns= ['RecordType', 'StaffNumber','FleetCode','Rank','COPReportDate','COPReportTime','COPStartDate','COPEndDate','COPEndTime','TurnStn','COPStartDay','COPType','FlightNumber','SpecialDuty','CrewRemarks'])
    roster=roster[roster['COPStartDate']!='']
    roster['COPReportDate']= pd.to_datetime(roster['COPReportDate'], format='%d/%m/%y')
    roster['COPStartDate']= pd.to_datetime(roster['COPStartDate'], format='%d/%m/%y')
    roster['COPEndDate']= pd.to_datetime(roster['COPEndDate'], format='%d/%m/%y')
    roster['Rank']=roster['Rank'].str.strip()
    roster['TurnStn']=roster['TurnStn'].str.strip()
    roster['COPType']=roster['COPType'].str.strip()
    roster['FlightNumber']=roster['FlightNumber'].str.strip()
    roster['TurnStn'] = roster['TurnStn'].apply(lambda x: x.upper())
    roster['CrewRemarks'] = roster['CrewRemarks'].apply(lambda x: x.upper())

    roster= roster[(roster['COPReportDate'] >= start_date)& (roster['COPReportDate'] < end_date)].reset_index(drop=True)
                
    roster['COPReportDate']= pd.to_datetime(roster['COPReportDate'], format='%d/%m/%y').dt.date
    roster['StaffNumber']=roster['StaffNumber'].astype(np.int64)
    roster=roster[(roster['COPType']!='SN')&(roster['COPType']!='CT')]

    roster=roster[(roster['TurnStn']=='SS55')|(roster['TurnStn']=='SS33')|(roster['TurnStn']=='SS88')|(roster['TurnStn']=='SS77')]

    return (roster)
 
""" Process Pairing File/DateCOP """
def PlannedDATECOPProcessing():
    
    
     
    strfilepath=''
    strDateCOP=''
    lstCOP=[]
    lstMonthlyCOP=[]
    dfPlannedData=pd.DataFrame(columns= ['RecordType', 'StaffNumber','FleetCode','Rank','COPReportDate','COPReportTime','COPStartDate','COPEndDate','COPEndTime','TurnStn','COPStartDay','COPType','FlightNumber','SpecialDuty','CrewRemarks'])
    

    collist=['Report Date','CS','FS','FSS','IFS','LS','LSS']
    
    dfCOPData=pd.DataFrame(columns=['Report Date','IFS','CS','LS','LSS','FS','FSS'])
    
    print(os.listdir())
    for item in os.listdir():
        print('Printing DateCOP files..')

#         start_date=dt.date(2018,9,1)
#         end_date=dt.date(2019,2,1)
        a=item[6:11]
        start_date=pd.to_datetime(a,format='%b%y').date()
        end_date = start_date + relativedelta(months=+1)
        for key in os.listdir(item):
            if 'DATECOP' in key.upper():
                          
                print('Printing DateCOP files..')

                strfilepath = os.path.join(item+'/'+key)
                lstCOP=[]
                f=open(strfilepath,'r')
                strDateCOP=f.read()
                strDateCOP=strDateCOP.split('\n4\n')
                with open(ProcessLogFileName, 'a') as logFileHandle:
                    
                    logFileHandle.write(str(dt.datetime.now())+' :   '+'File Read is:'+ key+'\n')  
                for x in range(0,len(strDateCOP)):
                    lstCOP.append(strDateCOP[x].split('\n'))
                lstCOP=lstCOP[:-1]
                if lstCOP[0][0].startswith('0'):
                    lstCOP[0]=lstCOP[0][1:]
                
                lstMonthlyCOP=[]

    # Extracting Date for Respective Months   

                for x in range(0,len(lstCOP)):                  

                    if (pd.to_datetime(lstCOP[x][0][1:9],format="%d/%m/%y").date() >= start_date)& (pd.to_datetime(lstCOP[x][0][1:9],format="%d/%m/%y").date()< end_date):
                        
                        lstMonthlyCOP.append(lstCOP[x])

                    else:
                        pass

    ### Ignore SN/CT crew            

                lstMonthlyCOP=[s for s in lstMonthlyCOP if s[0][23:25] not in ('SN','CT')]

                lstcrewcomp=[]
                for x in range(0,len(lstMonthlyCOP)):
                    for y in lstMonthlyCOP[x]:
                        if y[0]=='3':
                            lstcrewcomp.append((lstMonthlyCOP[x][0][1:9])+lstMonthlyCOP[x][1][21:25]+y[1:])

 ## get Crew Complement
                for y in lstcrewcomp:
                    y=y.strip()
                    dictData={}
                    dictData['Report Date']=y[0:8]
           
                    dictData['Time']=y[8:12]
                    
                    for x in range(12,len(y),5):
                        strcrew = y[x:x+5]
                        if strcrew=='':
                            pass
                        else:
                            strrank=strcrew[0:3].strip()
                            count=strcrew[3:5]
                            dictData[strrank]=count

                    dfData=pd.DataFrame(dictData,index=[0])
                    dfCOPData=pd.concat([dfCOPData,dfData],axis=0).reset_index(drop=True)
                with open(ProcessLogFileName, 'a') as logFileHandle:
                    
                    logFileHandle.write(str(dt.datetime.now())+' :   '+'Crew Complement Extraction succesfull :'+ key+'\n')
            
            elif 'ROSTER' in key.upper():
                strfilepath = os.path.join(item+'/'+key)
                with open(ProcessLogFileName, 'a') as logFileHandle:
                    
                    logFileHandle.write(str(dt.datetime.now())+' :   '+'File Read is:'+ key+'\n')
                dfPlannedRosterData=PlannedRoster(strfilepath,start_date,end_date)
                
                dfPlannedData=pd.concat([dfPlannedData,dfPlannedRosterData],axis=0)

    dfCOPData=dfCOPData.fillna(0)
    dfPlannedData=dfPlannedData.fillna(0)

    dfCOPData['Report Date']=pd.to_datetime(dfCOPData['Report Date'],format='%d/%m/%y')
    dfCOPData['Time']=pd.to_datetime(dfCOPData['Time'],format='%H%M')
    dfCOPData['Time']=dfCOPData['Time'].dt.time

    types=['SSS1','SSS2','SSS3','SSS0','SS22','SS80','SS60','SS33_55_88','SS33','SS55','SS88','SS77']


    for i in types:

        if i=="SSS1":

            df_SSS1=pd.DataFrame(columns=['Report Date','SSS1_IFS','SSS1_CS','SSS1_LS','SSS1_LSS','SSS1_FS','SSS1_FSS'])
            start_time= dt.time(0)
            end_time=dt.time(23,59)
            sby='SSS1_'
            df_SSS1=df_SSS1.append(datafilter(dfCOPData,start_time,end_time,sby))


        elif i=='SSS2':
            df_SSS2=pd.DataFrame(columns=['Report Date','SSS2_IFS','SSS2_CS','SSS2_LS','SSS2_LSS','SSS2_FS','SSS2_FSS'])
            start_time=dt.time(0)
            end_time=dt.time(11,59)
            sby='SSS2_'
            df_SSS2=df_SSS2.append(datafilter(dfCOPData,start_time,end_time,sby))


        elif i=='SSS3':
            df_SSS3=pd.DataFrame(columns=['Report Date','SSS3_IFS','SSS3_CS','SSS3_LS','SSS3_LSS','SSS3_FS','SSS3_FSS'])
            start_time=dt.time(12)
            end_time=dt.time(23,59)
            sby='SSS3_'
            df_SSS3=df_SSS3.append(datafilter(dfCOPData,start_time,end_time,sby))
            
        elif i=='SSS0':
            df_SSS0=pd.DataFrame(columns=['Report Date','SSS0_IFS','SSS0_CS','SSS0_LS','SSS0_LSS','SSS0_FS','SSS0_FSS'])
            start_time=dt.time(6,1)
            end_time=dt.time(23,59)
            sby='SSS0_'
            df_SSS0=df_SSS0.append(datafilter(dfCOPData,start_time,end_time,sby))
            
        elif i=='SS22':
            df_SS22=pd.DataFrame(columns=['Report Date','SS22_IFS','SS22_CS','SS22_LS','SS22_LSS','SS22_FS','SS22_FSS'])
            start_time=dt.time(0)
            end_time=dt.time(21,59)
            sby='SS22_'     
            df_SS22=df_SS22.append(datafilter(dfCOPData,start_time,end_time,sby))
            
        elif i=='SS80':
            df_SS80=pd.DataFrame(columns=['Report Date','SS80_IFS','SS80_CS','SS80_LS','SS80_LSS','SS80_FS','SS80_FSS'])
            start_time=dt.time(8)
            end_time=dt.time(23,59)
            sby='SS80_'
            df_SS80=df_SS80.append(datafilter(dfCOPData,start_time,end_time,sby))

        elif i=='SS60':
            df_SS60=pd.DataFrame(columns=['Report Date','SS60_IFS','SS60_CS','SS60_LS','SS60_LSS','SS60_FS','SS60_FSS'])
            start_time=dt.time(6)
            end_time=dt.time(23,59)
            sby='SS60_'
           
            df_SS60=df_SS60.append(datafilter(dfCOPData,start_time,end_time,sby))
        
        elif i=='SS33_55_88':
            
            dfPlanRoster_new=pd.pivot_table(dfPlannedData[['COPReportDate','Rank']],index=['COPReportDate'],columns=['Rank'],aggfunc=len).reset_index()
            dfPlanRoster_new['SS33_55_88_SUM']= dfPlanRoster_new.sum(axis=1)
            dfPlanRoster_new=dfPlanRoster_new.rename(columns={'COPReportDate':'Report Date','IFS':'SS33_55_88_IFS','CS':'SS33_55_88_CS','LS':'SS33_55_88_LS','LSS':'SS33_55_88_LSS','FS':'SS33_55_88_FS','FSS':'SS33_55_88_FSS'})
            print(dfPlanRoster_new)

        elif (i=='SS33')|(i=='SS55')|(i=='SS88')|(i=='SS77'):
            
            df_Pln_Roster=pd.pivot_table(dfPlannedData[['COPReportDate','Rank','TurnStn']],index=['COPReportDate'],columns=['TurnStn','Rank'],aggfunc=len).reset_index()
            df_Pln_Roster.columns = ["_".join((str(i),str(j))) for i,j in df_Pln_Roster.columns]
            col_ss33=['SS33_CS','SS33_FS','SS33_FSS','SS33_IFS','SS33_LS','SS33_LSS']
            df_Pln_Roster['SS33_SUM']= df_Pln_Roster[col_ss33].sum(axis=1)
            col_ss55=['SS55_CS','SS55_FS','SS55_FSS','SS55_IFS','SS55_LS','SS55_LSS']
            df_Pln_Roster['SS55_SUM']= df_Pln_Roster[col_ss55].sum(axis=1)
            col_ss88=['SS88_CS','SS88_FS','SS88_FSS','SS88_IFS','SS88_LS','SS88_LSS']
            df_Pln_Roster['SS88_SUM']= df_Pln_Roster[col_ss88].sum(axis=1)
            col_ss77=['SS77_CS','SS77_FS','SS77_FSS','SS77_IFS','SS77_LS','SS77_LSS']
            df_Pln_Roster['SS77_SUM']= df_Pln_Roster[col_ss77].sum(axis=1)
            dfPlanRoster=df_Pln_Roster.rename(columns={'COPReportDate_':'Report Date'})


    df_new=[df_SSS1,df_SSS2,df_SSS3,df_SSS0,df_SS22,df_SS80,df_SS60,dfPlanRoster_new, dfPlanRoster]
    df_final = reduce(lambda left,right: pd.merge(left,right,on='Report Date',how='outer'), df_new)

    colname=list(df_final.columns.values)
    column_=[x for x in colname if not x.startswith('Report Date')]
    df_DATECOP = df_final[column_].add_prefix('PCOP_')
    df_DATECOP['Report Date'] = df_final['Report Date'].copy()

    cols = ['Day', 'Month', 'Year', 'DayOfWeek','WeekofMonth', 'WeekOfYear','isWeekend','isHoliday','HolidayDesc']
    df=pd.DataFrame(columns=cols)

    for key in os.listdir():
        a=key[6:11]
#         start_date=dt.date(2018,11,1)
#         end_date=dt.date(2019,2,1)

        start_date=pd.to_datetime(a,format='%b%y').date()
        end_date = start_date + relativedelta(months=+1)
        delta = end_date - start_date
        currDate = start_date
        while (currDate < end_date):
            df.loc[currDate] = {
            'Day':currDate.day, 'Month':currDate.month, 'Year':currDate.year, 
            'DayOfWeek':currDate.isoweekday(),'WeekofMonth':weekOfMonth(currDate),'WeekOfYear':currDate.isocalendar()[1],   
            'isWeekend':isWeekend(currDate), 'isHoliday':0,'HolidayDesc':""}
            currDate = currDate + dt.timedelta(days=1)

    dfHoliday = pd.read_csv('D:/sa/rosterh_last/SIA_PublicHoliday_20152017.csv',encoding='cp1252')
    with open(ProcessLogFileName, 'a') as logFileHandle:
        logFileHandle.write(str(dt.datetime.now())+' :   '+'File read is Holiday Desc'+'\n')
    dfHoliday['Date'] = pd.to_datetime(dfHoliday['Date'],format='%m/%d/%Y',errors='coerce')

    for i in range(0, len(dfHoliday)):

        if dfHoliday.iloc[i]['Date'].date() in df.index:

            df.loc[dfHoliday.iloc[i]['Date'].date(), 'isHoliday']   = 1
            df.loc[dfHoliday.iloc[i]['Date'].date(), 'HolidayDesc'] = dfHoliday.iloc[i]['Remark']

    df.reset_index(level=0, inplace=True)
    df['index']=pd.to_datetime(df['index'],format='%Y-%m-%d')
    
    df_DATECOP_new=df.merge(df_DATECOP,left_on=['index'],right_on=['Report Date'],how='inner')
    with open(ProcessLogFileName, 'a') as logFileHandle:
        logFileHandle.write(str(dt.datetime.now())+' :   '+'DATECOP PROCESSED'+'\n')

    return(df_DATECOP_new)
        
""" filter data by Standby codes"""
def datafilter(dfCOPData,start_time,end_time,sby):
    collist=['Report Date','CS','FS','FSS','IFS','LS','LSS']
    dfCOPData = dfCOPData[(dfCOPData['Time'] >= start_time) & (dfCOPData['Time'] <= end_time)]
    for column in collist:
        if dfCOPData[column].dtypes=='object':
            dfCOPData[column]=dfCOPData[column].astype(np.int64)
    
    dfPCOP=pd.pivot_table(dfCOPData[collist],index=['Report Date'],aggfunc=sum).reset_index()
    dfPCOP['Sum']=dfPCOP.sum(axis=1) 
    colnames=list(dfPCOP.columns.values)
 
    column_new=[x for x in colnames if not x.startswith('Report Date')]
    
    PlannedDATECOP = dfPCOP[column_new].add_prefix(sby)  

    PlannedDATECOP['Report Date'] = dfPCOP['Report Date'].copy()

    return(PlannedDATECOP)   

"""Process Actual Roster """
def ProcessRoster(filepath):
    
    roster = {}
    filenames = []
    
    dfActRoster = pd.DataFrame(columns= ['RecordType', 'StaffNumber','FleetCode','Rank','COPReportDate','COPReportTime','COPStartDate','COPEndDate','COPEndTime','TurnStn','COPStartDay','COPType','FlightNumber','SpecialDuty','CrewRemarks'])
    dfActRoster = dfActRoster.fillna(0)
    
    for file in os.listdir(filepath):
        try:
#             print('Printing Actual Roster files..')
            filenames.append(file) 
            roster[file]= np.genfromtxt(filepath+file,delimiter=[1,8,3,3,8,4,8,8,4,5,1,2,8,6,9,3,20],usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,15,16),skip_header= True,skip_footer=True,dtype=str)
            roster[file]= pd.DataFrame(roster[file], columns= ['RecordType', 'StaffNumber','FleetCode','Rank','COPReportDate','COPReportTime','COPStartDate','COPEndDate','COPEndTime','TurnStn','COPStartDay','COPType','FlightNumber','SpecialDuty','CrewRemarks'])
            roster[file]=roster[file][roster[file]['COPStartDate']!='']
            roster[file]['COPReportDate']= pd.to_datetime(roster[file]['COPReportDate'], format='%d/%m/%y')
            roster[file]['COPStartDate']= pd.to_datetime(roster[file]['COPStartDate'], format='%d/%m/%y')
            roster[file]['COPEndDate']= pd.to_datetime(roster[file]['COPEndDate'], format='%d/%m/%y')
            roster[file]['Rank']=roster[file]['Rank'].str.strip()
            roster[file]['TurnStn']=roster[file]['TurnStn'].str.strip()
            roster[file]['COPType']=roster[file]['COPType'].str.strip()
            roster[file]['FlightNumber']=roster[file]['FlightNumber'].str.strip()
            roster[file]['TurnStn'] = roster[file]['TurnStn'].apply(lambda x: x.upper())
            roster[file]['CrewRemarks'] = roster[file]['CrewRemarks'].apply(lambda x: x.upper())
            
            with open(ProcessLogFileName, 'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+' :  '+'Extraction of Actual Roster  is succesfull :' + file + '\n')
        except(FileNotFoundError) as E:
            with open(ProcessLogFileName,'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+': '+str(E)+'\n')
    for key in roster.keys():
        if key[11:13]=='01':
            roster[key] = roster[key][(roster[key]['COPReportDate'] >= str(int(key[7:11])-1)+'-12-01') & (roster[key]['COPReportDate'] < key[7:11]+'-'+str(int(key[11:13])).zfill(2)+'-01' ) ].reset_index(drop=True)
        else:
            roster[key] = roster[key][(roster[key]['COPReportDate'] >= key[7:11]+'-'+str(int(key[11:13])-1).zfill(2)+'-01') & (roster[key]['COPReportDate'] < key[7:11]+'-'+str(int(key[11:13])).zfill(2)+'-01' )].reset_index(drop=True)

        dfActRoster=dfActRoster.append(roster[key])
    
    dfActRoster['COPReportDate']= pd.to_datetime(dfActRoster['COPReportDate'], format='%d/%m/%y').dt.date
    dfActRoster['StaffNumber']=dfActRoster['StaffNumber'].astype(np.int64)
    
    dfActRoster=dfActRoster[(dfActRoster['COPType']!='SN')&(dfActRoster['COPType']!='CT')]
    
                
    return(dfActRoster)

""" Process RosterH files """
def ProcessRosterH(sbyfilepath,rosterhfilepath):
    
    df_rostH={}
    dfRosterH = pd.DataFrame(columns= ['RecordType','StaffNumber','RosterStart','RosterEnd','DutyType','Status','Timestamp','DutyCode'])
    dfRosterH = dfRosterH.fillna(0)

    for item in os.listdir(sbyfilepath):
        try:
            df_standby=pd.read_excel(sbyfilepath+item,sheetname='Data')
            
            with open(ProcessLogFileName, 'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+' :   '+'File read is  :'+ item +'\n')
        
            df_standby['Month']= pd.to_datetime(df_standby['Month'], format='%Y%m%d%H%M').dt.date
            df_standby['Mon']= pd.to_datetime(df_standby['Month'], format='%Y-%m-%d').dt.month
            df_standby['Year']= pd.to_datetime(df_standby['Month'], format='%Y-%m-%d').dt.year
            df_standby['New']=(df_standby['Year']).map(str)+(df_standby['Mon']).map(str)
        
        except(FileNotFoundError) as E:
            with open(ProcessLogFileName,'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+': '+str(E)+'\n')
        
        
    for file in os.listdir(rosterhfilepath):
        print(file)
        try:
            df_rostH[file]=pd.read_csv(rosterhfilepath+file,names =['RecordType','StaffNumber','RosterStart','RosterEnd','DutyType','Status','Timestamp','DutyCode','StaffNumber_new'],dtype=str, sep='\t|,', engine='python')
                       
                      
            with open(ProcessLogFileName, 'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+':  '+'File read is : '+ file+'\n')
            df_rostH[file]=df_rostH[file].drop(['Status'],axis=1)
            

            
            df_rostH[file]['RosterStart']=pd.to_datetime(df_rostH[file]['RosterStart'], format='%Y%m%d%H%M')+ timedelta(hours=8)
            df_rostH[file]['RosterStart']= pd.to_datetime(df_rostH[file]['RosterStart'], format='%Y%m%d%H%M').dt.date

            df_rostH[file]=df_rostH[file].dropna(axis=0, how= 'any')

            df_rostH[file]['Timestamp']= pd.to_datetime(df_rostH[file]['Timestamp'], format='%Y%m%d%H%M')+timedelta(hours=8)
            df_rostH[file]=df_rostH[file].drop_duplicates(subset=['RosterStart','StaffNumber','Timestamp'],keep='first',inplace=False)

            df_rostH[file]['Month']= pd.to_datetime(df_rostH[file]['RosterStart'], format='%Y-%m-%d').dt.month
            df_rostH[file]['Year']= pd.to_datetime(df_rostH[file]['RosterStart'], format='%Y-%m-%d').dt.year
            df_rostH[file]['New']=(df_rostH[file]['Year']).map(str)+(df_rostH[file]['Month']).map(str)
            df_rostH[file]['Timestp']=pd.to_datetime(df_rostH[file]['Timestamp'], format='%Y-%m-%d').dt.date

            dfRosterH=dfRosterH.append(df_rostH[file])
        except(FileNotFoundError) as E:
            with open(ProcessLogFileName, 'a') as logFileHandle:
                logFileHandle.write(str(dt.datetime.now())+': '+str(E)+'\n')
    
    dfRosterHfilter=dfRosterH.merge(df_standby,left_on='New',right_on='New')
    
    with open(ProcessLogFileName, 'a') as logFileHandle:
        logFileHandle.write(str(dt.datetime.now())+':  '+'Rosterh and standby file merged successfully  '+'\n')
    dfRosterHfilter['Publish Date']=pd.to_datetime(dfRosterHfilter['Publish Date'], format='%Y-%m-%d').dt.date
    dfRosterHfilter=dfRosterHfilter[dfRosterHfilter['Timestp']>dfRosterHfilter['Publish Date']]
    
    dfRosterHpivot=pd.pivot_table(dfRosterHfilter,index=['RosterStart','StaffNumber'],values=['Timestamp'],aggfunc=max).reset_index()
        
    dfRosterHpivot=dfRosterHpivot.merge(dfRosterHfilter,left_on=['RosterStart','StaffNumber','Timestamp'],right_on=['RosterStart','StaffNumber','Timestamp'])
    
    dfRosterHpivot=dfRosterHpivot.loc[dfRosterHpivot['DutyType'].isin(['SBY'])]
    
    return(dfRosterHpivot)

# get standby numbers
dfActRoster_2016=ProcessRoster(Roster2016)
dfActRoster_2017=ProcessRoster(Roster2017)
dfActRoster_2018=ProcessRoster(Roster2018)

dfActRoster=dfActRoster_2016.append([dfActRoster_2017,dfActRoster_2018])

dfRosterH_2016=ProcessRosterH(SBYRequired,RosterH2016)
dfRosterH_2017=ProcessRosterH(SBYRequired,RosterH2017)
dfRosterH_2018=ProcessRosterH(SBYRequired,RosterH2018)


dfRosterH=dfRosterH_2016.append([dfRosterH_2017,dfRosterH_2018])



#####  if staff_number is masked

# crew_file='D:/sa/EIM/CC Crew.xlsx'
# df_crewmask=pd.read_excel(crew_file)

# df_crewmask['Original']=df_crewmask['Original'].map(str)

# dfRosterH=dfRosterH.merge(df_crewmask,left_on='StaffNumber',right_on='Original',how='inner')

dfRosterH['new']=dfRosterH['RosterStart'].map(str)+dfRosterH['StaffNumber'].map(str)

dfActRoster['new']=dfActRoster['COPReportDate'].map(str)+dfActRoster['StaffNumber'].map(str)

dfActRoster=dfActRoster[dfActRoster['FlightNumber']!='']


dfActRoster=dfActRoster.merge(dfRosterH, on=['new'],how='inner')


with open(ProcessLogFileName, 'a') as logFileHandle:
    logFileHandle.write(str(dt.datetime.now())+':  '+'Rosterh and Actual Roster merged successfully  '+'\n')



dfActRoster=dfActRoster.drop_duplicates(keep='first',inplace=False)

# get rankwise standby utilisation
df_rank=pd.pivot_table(dfActRoster[['COPReportDate','Rank']],index=['COPReportDate'], columns=['Rank'],aggfunc=len).reset_index()

col=list(df_rank.columns)

col_new=[x for x in col if not x.startswith('COPReportDate')]

# get standby type utilisation
df_dutycode=pd.pivot_table(dfActRoster[['COPReportDate','DutyCode']],index=['COPReportDate'], columns=['DutyCode'],aggfunc=len).reset_index()

col1=list(df_dutycode.columns)

col_new1=[x for x in col1 if not x.startswith('COPReportDate')]

col_list= list(df_dutycode[['SSS1','SSS2','SSS3']])

 
df_dutycode['ASBCU_SSS1-3']=df_dutycode[col_list].sum(axis=1)
df_count_new=pd.pivot_table(dfActRoster[['COPReportDate','Rank','DutyCode']],index=['COPReportDate'],columns=['DutyCode','Rank'],aggfunc=len).reset_index()

# get Standby type by Rank utilisation
lstSS=['SSS1','SS80','SS55','SS33','SSS2','SSS3','SS22','SS88','SS60','SS77','SSYY','SSS0','SSSI','SSSA','SSSJ','SSSQ','SSSH','SSSB','SSSC','STBY','SSSF','SSSD']
Rank=['IFS','CS','FS','FSS','LS','LSS']
SS_Rank=lstSS+Rank


for i in lstSS:
    for j in Rank:
        SS_Rank.append("_".join((str(i),str(j))))

df_count_new.columns = ["_".join((str(i),str(j))) for i,j in df_count_new.columns]
 
df_count_new=df_count_new.rename(columns={'COPReportDate_':'COPReportDate'})
collst= df_count_new.columns
collst_new= [x for x in collst if not x.startswith('COPReportDate')]

col_list2=list(df_count_new[['SSS1_IFS','SSS1_CS','SSS2_IFS','SSS2_CS','SSS3_IFS','SSS3_CS']])
df_count_new['SSS1-3CS_IFS']=df_count_new[col_list2].sum(axis=1)
col_list3=list(df_count_new[['SSS1_LS','SSS1_LSS','SSS2_LS','SSS2_LSS','SSS3_LS','SSS3_LSS']])
df_count_new['SSS1-3LS_LSS']=df_count_new[col_list3].sum(axis=1)
col_list4=list(df_count_new[['SSS1_FS','SSS1_FSS','SSS2_FS','SSS2_FSS','SSS3_FS','SSS3_FSS']])
df_count_new['SSS1-3FS_FSS']=df_count_new[col_list4].sum(axis=1)


df_count_new['ASBCU_SSS1-3']=df_dutycode['ASBCU_SSS1-3'].copy()

df_count_new['SSS1-3CS_IFSRatio']=df_count_new['SSS1-3CS_IFS']/df_count_new['ASBCU_SSS1-3']
df_count_new['SSS1-3LS_LSSRatio']=df_count_new['SSS1-3LS_LSS']/df_count_new['ASBCU_SSS1-3']
df_count_new['SSS1-3FS_FSSRatio']=df_count_new['SSS1-3FS_FSS']/df_count_new['ASBCU_SSS1-3']

df_count_new=np.round(df_count_new, decimals=2)

df_count_final=pd.DataFrame(columns=SS_Rank)

df_merge=df_rank.merge(df_dutycode,on='COPReportDate',how='outer')
with open(ProcessLogFileName, 'a') as logFileHandle:
    logFileHandle.write(str(dt.datetime.now())+':  '+'Rank and dutycode  merged successfully  '+'\n')
df_merge=df_merge.merge(df_count_new,on='COPReportDate',how='outer')
with open(ProcessLogFileName, 'a') as logFileHandle:
    logFileHandle.write(str(dt.datetime.now())+':  '+'Rank, dutycode and Dutycode_rank  merged successfully  '+'\n')
    

lstcol=df_merge.columns

df_merge_new=df_merge.drop(['ASBCU_SSS1-3_y'],axis=1)

common_cols=list(set(lstcol).intersection(set(df_count_final.columns)))

A_cols=list(set(df_merge_new.columns).difference(set(common_cols)))

new_cols=common_cols+A_cols

df_count_final[new_cols]=df_merge_new[new_cols]

df_count_final=df_count_final.rename(columns={'ASBCU_SSS1-3_x':'ASBCU_SSS1-3'})

newlist=[]
newlist=lstSS+Rank
SS_Rank_new=list(set(SS_Rank).difference(set(newlist)))

df_count_final['ASBCU_SUM']=df_count_final[SS_Rank_new].sum(axis=1)

PlannedDatecop =PlannedDATECOPProcessing()

df_final=PlannedDatecop.merge(df_count_final,left_on='Report Date',right_on='COPReportDate',how='outer')
df_final=df_final.drop(['COPReportDate','Report Date'],axis=1)
df_final=df_final.rename(columns={'index':'COPReportDate'})
df_final.to_csv('D:/sa/output/sample_fil.csv',index=False)

with open(ProcessLogFileName, 'a') as logFileHandle:
    logFileHandle.write(str(dt.datetime.now())+':  '+' historical data generated '+'\n')
